### ДЗ2

Хранение расписания тренировок с детализацией упражнений по каждой тренировке по каждому участнику.

### Подготовка данных для создания БД

**Участники тренировок**

In [1]:
import pandas as pd
import random
from random import sample

In [2]:
russian_names = pd.read_csv('russian_names.csv', sep=';')

In [3]:
random_names = russian_names.Name.sample(n=5)

In [4]:
names = random_names.tolist()

In [5]:
names

['Фарадиса', 'Домиан', 'Хадира', 'Агонета', 'Кушан']

**Дни недели**

In [6]:
days = ['Понедельник', 'Вторник', 'Среда', 'Четверг', 'Пятница', 'Суббота', 'Воскресенье']

**Пусть каждый участник посещает как минимум 1 (но не больше 3-х) тренировок в неделю**

In [7]:
import copy

In [8]:
sportsmen_list = copy.copy(names)

In [9]:
sportsmen_list += ([i for i in names] * 2)

In [10]:
workout_days = []
for i in range(len(sportsmen_list)):
    workout_days.append(random.choice(days))

In [11]:
workout_days = list(set(tuple(zip(sportsmen_list, workout_days))))

In [12]:
workout_days[:10]

[('Агонета', 'Вторник'),
 ('Кушан', 'Суббота'),
 ('Кушан', 'Пятница'),
 ('Хадира', 'Суббота'),
 ('Домиан', 'Пятница'),
 ('Фарадиса', 'Четверг'),
 ('Фарадиса', 'Суббота'),
 ('Домиан', 'Понедельник'),
 ('Кушан', 'Среда'),
 ('Хадира', 'Вторник')]

**Тренеры**

In [13]:
trainers = random.sample(list(set(russian_names.Name.tolist()) - set(names)), len(sportsmen_list))

In [14]:
trainers[:5]

['Гранида', 'Лолитта', 'Фархат', 'Асмиральда', 'Нела']

**Тренировки**

In [15]:
with open('workout.txt', 'r', encoding='utf-8') as f:
    workouts = f.read().split('\n')

In [16]:
workouts

['Баттерфляй',
 'Бег',
 'Боковые выпады',
 'Боковые скручивания',
 'Брасс',
 'Жим гантелей лежа',
 'Махи ногой',
 'Отжимания',
 'Планка',
 'Подтягивание на перекладине']

### Создание БД

In [17]:
import psycopg2

In [18]:
conn = psycopg2.connect(dbname='bg3nkuldzalayon',
                       user='uu6gphexkx3fzzeb66mn',
                       host='bg3nkuldzalayon-postgresql.services.clever-cloud.com',
                       password='UOz4hO5b1ah86rNyqP48')

cur = conn.cursor()

**Создание таблиц**

In [19]:
# cur.execute('DROP TABLE sportsmen, trainers, workouts, days, schedule')

In [20]:
cur.execute('CREATE TABLE sportsmen (Name TEXT);')
cur.execute('CREATE TABLE trainers (Name TEXT);')
cur.execute('CREATE TABLE workouts (Workout TEXT);')
cur.execute('CREATE TABLE days (Day TEXT);')
cur.execute('CREATE TABLE schedule (Name TEXT, Workout TEXT, Day TEXT, Trainer TEXT);')

**Заполнение таблиц**

In [21]:
for element in names:
    cur.execute('INSERT INTO sportsmen VALUES (%s)', [element])
    
for element in trainers:
    cur.execute('INSERT INTO trainers VALUES (%s)', [element])
    
for element in workouts:
    cur.execute('INSERT INTO workouts VALUES (%s)', [element])
    
for element in days:
    cur.execute('INSERT INTO days VALUES (%s)', [element])

**Пусть тренировка каждого участника состоит из 3-х случайных упражнений**

In [22]:
workout = []
for i in range(len(sportsmen_list)):
    workout.append(', '.join(sample(workouts, 3)))

In [23]:
workout[0]

'Отжимания, Боковые выпады, Бег'

In [24]:
for element, w, tr in zip(workout_days, workout, trainers):
    cur.execute('INSERT INTO schedule VALUES (%s, %s, %s, %s);', [element[0], w, element[1], tr])

**Tables overview**

In [25]:
tables = ['sportsmen', 'trainers', 'workouts', 'days', 'schedule']

In [26]:
def show(tables):
    for table in tables:
        cur.execute('SELECT * FROM %s' % table)
        print(table.upper())
        for data in cur.fetchall()[:5]:
            print(data)

In [27]:
show(tables)

SPORTSMEN
('Фарадиса',)
('Домиан',)
('Хадира',)
('Агонета',)
('Кушан',)
TRAINERS
('Гранида',)
('Лолитта',)
('Фархат',)
('Асмиральда',)
('Нела',)
WORKOUTS
('Баттерфляй',)
('Бег',)
('Боковые выпады',)
('Боковые скручивания',)
('Брасс',)
DAYS
('Понедельник',)
('Вторник',)
('Среда',)
('Четверг',)
('Пятница',)
SCHEDULE
('Агонета', 'Отжимания, Боковые выпады, Бег', 'Вторник', 'Гранида')
('Кушан', 'Махи ногой, Подтягивание на перекладине, Боковые выпады', 'Суббота', 'Лолитта')
('Кушан', 'Подтягивание на перекладине, Жим гантелей лежа, Махи ногой', 'Пятница', 'Фархат')
('Хадира', 'Планка, Боковые выпады, Брасс', 'Суббота', 'Асмиральда')
('Домиан', 'Подтягивание на перекладине, Планка, Махи ногой', 'Пятница', 'Нела')


**Посмотрим на расписание тренировок какого-либо спортсмена**

In [28]:
def show_schedule(name):
    if name not in names:
        print('Такого имени нет в базе данных.')
        return None

    query = "SELECT * FROM schedule WHERE Name LIKE '%s'" % (name)
    cur.execute(query)
    sportsman = cur.fetchall()
    
    print(sportsman)

In [29]:
rand = random.choice(names)
print(rand.upper())
show_schedule(random.choice(names))

ДОМИАН
[('Домиан', 'Подтягивание на перекладине, Планка, Махи ногой', 'Пятница', 'Нела'), ('Домиан', 'Баттерфляй, Боковые выпады, Брасс', 'Понедельник', 'Эржибетта'), ('Домиан', 'Отжимания, Брасс, Жим гантелей лежа', 'Среда', 'Алетий')]


In [30]:
show_schedule('Имиромиваромвы')

Такого имени нет в базе данных.


In [31]:
def show_trainer_schedule(name):
    if name not in trainers:
        print('Такого имени нет в базе данных.')
        return None

    cur.execute("SELECT * FROM schedule WHERE schedule.Trainer LIKE '%s'" % (name))
    trainer = cur.fetchall()
        
    print(trainer)

In [32]:
rand = random.choice(trainers)
print(rand.upper())
show_trainer_schedule(rand)

НЕЛА
[('Домиан', 'Подтягивание на перекладине, Планка, Махи ногой', 'Пятница', 'Нела')]


**DELETE**

In [33]:
def check(table, j):
    cur.execute('SELECT * FROM %s' % table)
    rows = cur.fetchall()
    check_rows = []
    for i in range(len(rows)):
        check_rows.append(rows[i][j].lower())
    
    return check_rows

In [34]:
def delete_sportsman(name):
    if name.lower() not in check('sportsmen', j=0):
        print('Такого имени нет в базе данных.')
        
    else:
        cur.execute("DELETE FROM sportsmen WHERE Name = '%s'" % name)
        cur.execute("DELETE FROM schedule WHERE Name IN (SELECT Name FROM schedule WHERE Name = '%s')" % name)
        conn.commit()
        names.remove(name)
        print(name + ' was deleted.')

In [35]:
delete_sportsman('Fnjkfnf')

Такого имени нет в базе данных.


In [36]:
delete_sportsman(random.choice(names))

Фарадиса was deleted.


In [37]:
def delete_trainer(name):
    if name.lower() not in check('trainers', j=0):
        print('Такого имени нет в базе данных.')
    
    else:
        cur.execute("DELETE FROM trainers WHERE Name = '%s'" % name)
        # Чтобы участники тренировок и другие данные остались в таблице
        cur.execute("UPDATE schedule SET Trainer = (%s) WHERE Trainer IN \
                    (SELECT Trainer FROM schedule WHERE Trainer = '%s')" % ('null', name))
        conn.commit()
        trainers.remove(name)
        print(name + ' was deleted.')

In [38]:
delete_trainer('BFHbhjfbfjf')

Такого имени нет в базе данных.


In [39]:
delete_trainer(random.choice(trainers))

Резеда was deleted.


**INSERT**

In [40]:
def new_sportsman(name):
    if name.lower() in check('sportsmen', j=0):
        print('Такой участник уже есть в базе данных.')
        
    else:
        cur.execute('INSERT INTO sportsmen VALUES (%s)', [name])
        conn.commit()
        names.append(name)
        print(name + ' was added.')

In [41]:
new_sportsman('Катя')

Катя was added.


In [43]:
def new_trainer(name):
    if name.lower() in check('trainers', j=0):
        print('Такой тренер уже есть в базе данных.')
    
    else:
        cur.execute('INSERT INTO trainers VALUES (%s)', [name])
        conn.commit()
        trainers.append(name)
        print(name + ' was added.')

In [44]:
new_trainer('Владимир')

Владимир was added.


In [45]:
def new_exercise(name):
    if name.lower() in check('workouts', j=0):
        print('Такое упражнение уже есть в базе данных.')
    
    else:
        cur.execute('INSERT INTO workouts VALUES (%s)', [name])
        conn.commit()
        workouts.append(name)
        print(name + ' was added.')

In [46]:
new_exercise('Растяжка')

Растяжка was added.


In [47]:
def new_data(name, workout, day, trainer):
    cur.execute('INSERT INTO schedule VALUES (%s, %s, %s, %s)', (name, workout, day, trainer))
    conn.commit()
    print(name + ', ' + workout + ', ' + day + ', ' + trainer + ' were added.')

In [48]:
new_data('Оля', 'Прыжки, танцы, отжимания', 'Понедельник', 'Андрей')

Оля, Прыжки, танцы, отжимания, Понедельник, Андрей were added.


**UPDATE**

In [49]:
def update_sportsman(name1, name2):
    if name1.lower() not in check('sportsmen', j=0) and name1.lower() not in check('schedule', j=0):
        print('Такого участника нет в базе данных.')
    
    else:
        cur.execute("UPDATE sportsmen SET Name = %s WHERE Name = %s", (name2, name1))
        cur.execute("UPDATE schedule SET Name = %s WHERE Name = %s", (name2, name1))
        conn.commit()
        names[names.index(name1)] = name2
        print(name1 + ' was changed to ' + name2 + '.')

In [50]:
update_sportsman('Катя', 'Оля')

Катя was changed to Оля.


In [51]:
def update_trainer(name1, name2):
    if name1.lower() not in check('trainers', j=0):
        print('Такого участника нет в базе данных.')
    
    else:
        cur.execute("UPDATE trainers SET Name = %s WHERE Name = %s", (name2, name1))
        cur.execute("UPDATE schedule SET Trainer = %s WHERE Trainer = %s", (name2, name1))
        conn.commit()
        trainers[trainers.index(name1)] = name2
        print(name1 + ' was changed to ' + name2 + '.')

In [52]:
update_trainer('Тедбир', 'Игорь')

Такого участника нет в базе данных.


In [53]:
def update_day(name, day):
    cur.execute("UPDATE schedule SET Day = %s WHERE Name = %s", (day, name))
    conn.commit()
    print('Day for ' + name + ' was changed to ' + day + '.')

In [54]:
update_day('Игорь', 'Вторник')

Day for Игорь was changed to Вторник.


**JOIN**

Добавим еще несколько строчек в таблицу <code>schedule</code>, чтобы с помощью <code>JOIN</code> проверить, например, какие участники тренировок есть в <code>schedule</code>, но отсутствуют в <code>sportsmen</code>.

In [55]:
new_data('Галя', 'Фигурное катание, танцы, бег', 'Понедельник', 'Кристина')

Галя, Фигурное катание, танцы, бег, Понедельник, Кристина were added.


In [56]:
new_data('Юля', 'Лыжи, растяжка, бег', 'Вторник', 'Игорь')

Юля, Лыжи, растяжка, бег, Вторник, Игорь were added.


In [57]:
new_data('Сергей', 'Йога, гимнастика, бег', 'Среда', 'Александр')

Сергей, Йога, гимнастика, бег, Среда, Александр were added.


In [58]:
new_data('Елена', 'Йога, гимнастика, бег', 'Среда', 'Александр')

Елена, Йога, гимнастика, бег, Среда, Александр were added.


In [59]:
def sportsman_trainer():
    cur.execute("SELECT * FROM schedule FULL OUTER JOIN sportsmen USING (Name) WHERE sportsmen.Name IS NULL")
    rows = cur.fetchall()

    if rows != []:
        headers = ['Name', 'Workout', 'Day', 'Trainer']
        print('{:<10s} {:<30s} {:<15s} {:<10s}'.format(*headers))
        for list_ in rows:
            print('{:<10s} {:<30s} {:<15s} {:<10s}'.format(*list_))
    
    else:
        print('Все участники тренировок содержатся в обеих таблицах.')

In [60]:
sportsman_trainer()

Name       Workout                        Day             Trainer   
Сергей     Йога, гимнастика, бег          Среда           Александр 
Елена      Йога, гимнастика, бег          Среда           Александр 
Галя       Фигурное катание, танцы, бег   Понедельник     Кристина  
Юля        Лыжи, растяжка, бег            Вторник         Игорь     


Добавим то, что нашлось, в таблицу <code>sportsmen</code>.

In [61]:
# cur.execute("SELECT * FROM schedule FULL OUTER JOIN sportsmen USING (Name) WHERE sportsmen.Name IS NULL")
# for i in cur.fetchall():
#     new_sportsman(i[0])

**Реализация некоторых запросов пользователя**

In [62]:
def user_input():
    # Список таблиц БД
    user = input('Чтобы узнать, какие таблицы есть в базе данных, введите 1:' + '\n')
    
    if user == '' or user != '1':
        user = input('Похоже, что вы ввели пустой или неверный запрос. Попробуйте еще раз!' + '\n')
    
    if user == '1':
        print('\n' + 'Названия таблиц, содержащихся в базе данных: ')
        cur.execute("SELECT * FROM pg_catalog.pg_tables WHERE schemaname != 'pg_catalog' \
                    AND schemaname != 'information_schema'")
        
        for i in cur.fetchall():
            print(i[1])
    
    # Содержимое таблиц
    user = input('Чтобы узнать, какие данные содержат имеющиеся таблицы, введите 2:' + '\n')
    
    if user == '' or user != '2':
        user = input('Похоже, что вы ввели пустой или неверный запрос. Попробуйте еще раз:' + '\n')

    if user == '2':
        print('\n' + 'Данные, содержащиеся в таблицах: ')
        show(tables)
        
    # Расписание тренировок конкретного спорсмена
    user = input('Чтобы узнать, какие тренировки посещает спортсмен, введите 3:' + '\n')
    
    if user == '' or user != '3':
        user = input('Похоже, что вы ввели пустой или неверный запрос. Попробуйте еще раз:' + '\n')
        
    if user == '3':
        print('\n' + 'Список доступных имен: ' + ', '.join(names))
        user = input('\n' + 'Введите имя спортсмена:' + '\n')
    
        if user.title() in names:
            print('\n' + 'Расписание тренировок для спортсмена %s: ' % user.title())
            show_schedule(user.title())

        else:
            user = input('К сожалению, такого спортсмена нет в базе данных. ' + 
                         'Чтобы добавить спортсмена в базу данных, введите 0 или попробуйте другой запрос:' + '\n')
            
            if user != '0':
                if user.title() in names:
                    print('\n' + 'Расписание тренировок для спортсмена %s: ' % user.title())
                    show_schedule(user.title())
                    
            if user == '0':
                user = input('\n' + 'Введите имя спортсмена, которого хотели бы добавить в базу данных:' + '\n')
                new_sportsman(user.title())
                
                user = input('\n' + 'Чтобы добавить тренировки, день занятий и тренера для спортсмена, введите 5:' + '\n')
                
                if user == '' or user != '5':
                    user = input('Похоже, что вы ввели пустой или неверный запрос. Попробуйте еще раз:' + '\n')
                    
                if user == '5':
                    name = input('\n' + 'Введите имя спортсмена:' + '\n')
                    workout = input('\n' + 'Введите список упражнений:' + '\n')
                    day = input('\n' + 'Введите день недели:' + '\n')
                    trainer = input('\n' + 'Введите имя тренера:' + '\n')
                    new_data(name, workout, day, trainer)
                     
    # Расписание занятий конкретного тренера
    user = input('Чтобы узнать, какие занятия проводит тренер, введите 4:' + '\n')
    
    if user == '' or user != '4':
        user = input('Похоже, что вы ввели пустой или неверный запрос. Попробуйте еще раз:' + '\n')
    
    if user == '4':
        print('\n' + 'Список доступных имен: ' + ', '.join(trainers))
        user = input('\n' + 'Выберите тренера из списка и введите его имя:' + '\n')
    
        if user.title() in trainers:
            print('\n' + 'Расписание занятий для тренера %s: ' % user.title())
            show_trainer_schedule(user.title())

        else:
            user = input('К сожалению, такого тренера нет в базе данных. ' + 
                         'Чтобы добавить тренера в базу данных, введите 9 или попробуйте другой запрос:' + '\n')
            
            if user != '9':
                if user.title() in trainers:
                    print('\n' + 'Расписание занятий для тренера %s: ' % user.title())
                    show_trainer_schedule(user.title())
                    
            if user == '9':
                user = input('\n' + 'Введите имя тренера, которого хотели бы добавить в базу данных:' + '\n')
                new_trainer(user.title())
    
    # Удалить спортсмена и все данные о нем из базы данных
    user = input('Чтобы удалить конкретного спортсмена из базы данных, введите 6:' + '\n')
    
    if user == '' or user != '6':
        user = input('Похоже, что вы ввели пустой или неверный запрос. Попробуйте еще раз:' + '\n')
    
    if user == '6':
        print('\n' + 'Список доступных имен: ' + ', '.join(names))
        user = input('\n' + 'Выберите спортсмена из списка и введите его имя, чтобы удалить:' + '\n')
    
        if user.title() in names:
            delete_sportsman(user.title())
            conn.commit()

        else:
            user = input('К сожалению, такого спортсмена нет в базе данных. Попробуйте еще раз:' + '\n')
            
            if user.title() in names:
                delete_sportsman(user.title())
                
    # Проверить, какие участники тренировок есть в schedule, но отсутствуют в sportsmen
    user = input('Чтобы сравнить данные в таблицах SCHEDULE и SPORTSMEN, введите 7:' + '\n')
    
    if user == '' or user != '7':
        user = input('Похоже, что вы ввели пустой или неверный запрос. Попробуйте еще раз:' + '\n')
    
    if user == '7':
        print('\n' + 'Информация о данных спортсменах содержится в таблице SCHEDULE, но отсутствует в таблице SPORTSMEN:' 
              + '\n')
        sportsman_trainer()
        
        user = input('Чтобы добавить найденных спорстменов в таблицу SPORTSMEN, введите 10. '
                    + 'Для выхода введите q:' + '\n')
        
        if user == '10':
            cur.execute("SELECT * FROM schedule FULL OUTER JOIN sportsmen USING (Name) WHERE sportsmen.Name IS NULL")
            for i in cur.fetchall():
                new_sportsman(i[0])
                names.append(i[0])
        
        if user != '10':
            pass

In [64]:
user_input()

Чтобы узнать, какие таблицы есть в базе данных, введите 1:
1

Названия таблиц, содержащихся в базе данных: 
sportsmen
trainers
workouts
days
schedule
Чтобы узнать, какие данные содержат имеющиеся таблицы, введите 2:
2

Данные, содержащиеся в таблицах: 
SPORTSMEN
('Домиан',)
('Хадира',)
('Агонета',)
('Кушан',)
('Светлана',)
TRAINERS
('Гранида',)
('Лолитта',)
('Фархат',)
('Асмиральда',)
('Нела',)
WORKOUTS
('Баттерфляй',)
('Бег',)
('Боковые выпады',)
('Боковые скручивания',)
('Брасс',)
DAYS
('Понедельник',)
('Вторник',)
('Среда',)
('Четверг',)
('Пятница',)
SCHEDULE
('Агонета', 'Отжимания, Боковые выпады, Бег', 'Вторник', 'Гранида')
('Кушан', 'Махи ногой, Подтягивание на перекладине, Боковые выпады', 'Суббота', 'Лолитта')
('Кушан', 'Подтягивание на перекладине, Жим гантелей лежа, Махи ногой', 'Пятница', 'Фархат')
('Хадира', 'Планка, Боковые выпады, Брасс', 'Суббота', 'Асмиральда')
('Домиан', 'Подтягивание на перекладине, Планка, Махи ногой', 'Пятница', 'Нела')
Чтобы узнать, какие тренировк